In [3]:
import chromadb
from sentence_transformers import SentenceTransformer

# 本地執行 Hugging Face 模型（不需要 API Key）
model = SentenceTransformer("intfloat/multilingual-e5-large")

# 初始化 ChromaDB(使用本地端保存的資料庫)
chroma_client = chromadb.PersistentClient(path='./chroma_db')
my_collection = chroma_client.get_or_create_collection(name='my_collection')


In [6]:
# 轉換文本成向量
texts = ["今天天氣很好！","下雨天適合喝熱茶。"]
embeddings = model.encode(texts).tolist() # 轉換成 list 格式,符合 ChromaDB 的要求
for i, (text, embedding) in enumerate(zip(texts, embeddings)):
    my_collection.add(
        ids=[f"id_{i}"],
        documents=[text],
        embeddings=[embedding]
    )
print("✅ 已儲存本地計算的嵌入向量！")

✅ 已儲存本地計算的嵌入向量！
